In [31]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [33]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [14]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

In [15]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [16]:
print(X_train.shape)
print(X_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [18]:
X_train = X_train.reshape(60000, 28, 28, 1)
X_test = X_test.reshape(10000, 28, 28, 1)

In [19]:
# One-Hot Encoding:
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

# Print the new label
print(y_train_one_hot[0])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [20]:
# Build the CNN model
model = Sequential()
# Add model layers
model.add(Conv2D(64, kernel_size=3, activation = 'relu', input_shape=(28,28,1)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None))
model.add(Flatten())
model.add(Dense(10,activation='softmax'))

In [21]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
hist = model.fit(X_train,y_train_one_hot, validation_data=(X_test,y_test_one_hot), epochs=10)
print("The model has successfully trained")
model.save('mnist_model.h5')
print("Saving the model as mnist_model.h5")

Epoch 1/10
1875/1875 [==============================] - 95s 50ms/step - loss: 0.2252 - accuracy: 0.9493 - val_loss: 0.0886 - val_accuracy: 0.9742
Epoch 2/10
1875/1875 [==============================] - 89s 47ms/step - loss: 0.0726 - accuracy: 0.9775 - val_loss: 0.0699 - val_accuracy: 0.9786
Epoch 3/10
1875/1875 [==============================] - 91s 49ms/step - loss: 0.0559 - accuracy: 0.9823 - val_loss: 0.0598 - val_accuracy: 0.9825
Epoch 4/10
1875/1875 [==============================] - 95s 51ms/step - loss: 0.0462 - accuracy: 0.9853 - val_loss: 0.0660 - val_accuracy: 0.9794
Epoch 5/10
1875/1875 [==============================] - 91s 49ms/step - loss: 0.0371 - accuracy: 0.9882 - val_loss: 0.0704 - val_accuracy: 0.9811
Epoch 6/10
1875/1875 [==============================] - 90s 48ms/step - loss: 0.0301 - accuracy: 0.9901 - val_loss: 0.0809 - val_accuracy: 0.9822
Epoch 7/10
1875/1875 [==============================] - 91s 49ms/step - loss: 0.0287 - accuracy: 0.9905 - val_loss: 0.0750 -

In [23]:
score = model.evaluate(X_test, y_test_one_hot, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.09725716710090637
Test accuracy: 0.9836000204086304


In [28]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np
model = load_model('mnist_model.h5')
def predict_digit(img):
    #resize image to 28x28 pixels
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)
class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.x = self.y = 0
        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command =         self.classify_handwriting) 
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)
        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)
    def clear_all(self):
        self.canvas.delete("all")
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id() # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas
        im = ImageGrab.grab(rect)
        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')
    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')
app = App()
mainloop()